In [138]:
import numpy as np
import pandas as pd
import re
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from stop_words import get_stop_words

In [139]:
kickstarter = pd.read_csv("csv/ks-projects-201801.csv",sep=',')

In [140]:
kickstarter.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [141]:
len(kickstarter)

378661

In [142]:
kickstarter = kickstarter[0:10000]
kickstarter

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.00,0.00,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.00,2421.00,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.00,220.00,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.00,1.00,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.00,1283.00,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1050352242,Flowerfish Games,Video Games,Games,USD,2017-04-07,2500.0,2017-02-06 22:34:24,0.0,failed,0,US,0.00,0.00,2500.00
9996,1050355782,SPNIME: The Supernatural / Anime Fanbook,Art,Art,USD,2015-04-20,1350.0,2015-03-18 03:43:41,2391.0,successful,57,US,2391.00,2391.00,1350.00
9997,1050357610,Respect The Youth-Lady Cellie's first EP album...,World Music,Music,USD,2011-07-06,4000.0,2011-06-06 00:01:36,0.0,canceled,0,US,0.00,0.00,4000.00
9998,1050358230,frog: across the pond,Documentary,Film & Video,USD,2016-01-24,3000.0,2015-12-24 01:06:19,5293.0,successful,60,US,5293.00,5293.00,3000.00


In [143]:
kickstarter['category'] = kickstarter['category'].apply(lambda category: re.sub('[^A-Za-z]+', ' ', category.lower()))

In [144]:
kickstarter.isnull().sum()

ID                    0
name                  0
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged         102
usd_pledged_real      0
usd_goal_real         0
dtype: int64

In [145]:
clf = make_pipeline(
    TfidfVectorizer(stop_words=get_stop_words('en')),
    OneVsRestClassifier(SVC(kernel='linear', probability=True))
)

In [146]:
X = kickstarter[['category','goal']]
Y = kickstarter['pledged']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
len(X_train)

8000

In [151]:
Y_train.dtypes

pledged    float64
dtype: object

In [147]:
X_train = X_train.transpose()

In [148]:
X_train.shape

(2, 8000)

In [149]:
Y_train.shape

(8000, 1)

In [150]:
clf = clf.fit(X=X_train, y=Y_train)

AttributeError: 'int' object has no attribute 'lower'